In [1]:
import numpy as np
import pandas as pd
import string
from sklearn.feature_extraction.text import CountVectorizer
from nltk.tokenize import RegexpTokenizer

In [2]:
def word_occurence_matrix(text, target=None, stop_words=None, binary=True, preprocess_text=False):  
    '''
    Output is messages x (unique) words
    
    If binary=True, then each element represents if the word is in the message or not.
    Otherwise, it represents the count of how many times that word appears in that message.
    ''' 
    if target:
        text = list(filter(lambda x : target in x, text)) #Filter comments in which target word is present
        
    preprocessor = CountVectorizer(strip_accents='unicode').build_preprocessor()   
    if stop_words:        
        stop_words = [preprocessor(word) for word in stop_words] #preprocesses stop words
    if preprocess_text:
        text = [preprocessor(msg) for msg in text] #preprocesses text
        
    #calculates word count for each message
    vectorizer = CountVectorizer(strip_accents='unicode', stop_words=stop_words, binary=binary)
    X = vectorizer.fit_transform(text).toarray()
    
    labels = vectorizer.get_feature_names()
    
    return X, labels

### Dados

In [3]:
comments = pd.read_csv('../../dados/youtube/comentarios_sorted_votes.csv')
stop_words = [word.rstrip() for word in open('../stopwords.txt')]

#Preprocesses text
preprocessor = CountVectorizer(strip_accents='unicode').build_preprocessor() #lowercase and strip accents
stop_words = [preprocessor(word) for word in stop_words]
comments['text'] = [preprocessor(msg) for msg in comments['text']]
comments['text'] = [' '.join([word for word in RegexpTokenizer(r'\w+').tokenize(msg) if not word in stop_words])
                    for msg in comments['text']]

N = 30 #Consider only the N most frequent words

In [4]:
comments_by_channel = []
channels = []
for channel, group in comments.groupby('uploader'):
    channel_comments = ' '.join(group['text'])
    comments_by_channel.append(channel_comments)
    channels.append(channel)

### Frequencia relativa

#### 1. Em relação ao total
Aqui foi usada a distribuição de probabilidade conjunta das palavras e canais, i.e., $p(a_i, c_j) = P(A=a_i, C= C_j)$.

In [27]:
X, labels = word_occurence_matrix(comments_by_channel, stop_words=stop_words, binary=False)

X = X / X.sum()

freq_lists = [list(zip(row,labels)) for row in X]
freq_lists = [sorted(channel, reverse=True) for channel in freq_lists]
df_freq = pd.DataFrame(freq_lists, index = channels).T

df_freq.head(10)

,Dr. Alain Dutra,Dr. Alvaro Galvão,Dr. Felipe Ades MD PhD,Dr. Fernando Gomes,Dr. Lair Ribeiro Oficial,Drauzio Varella,Julio Pereira - Neurocirurgião,Lucy Kerr
0,"(0.0019417820455064944, dias)","(0.000550356511183555, dias)","(0.00010319184584691656, dias)","(0.00028960292221554, dr)","(0.0005880825623533955, lair)","(0.003350961015674065, drauzio)","(0.0051939895742948005, dias)","(0.002137069839797433, ivermectina)"
1,"(0.0014014118419855442, tomar)","(0.00045937015247982213, dr)","(5.880825623533954e-05, dia)","(0.00025187687104569954, fernando)","(0.0005681099470281858, dr)","(0.0032433308108659902, pra)","(0.0035307145536009514, cheiro)","(0.0014857406622475406, dias)"
2,"(0.0013448227652307835, ivermectina)","(0.00042164410130998166, ivermectina)","(5.880825623533954e-05, covid)","(0.00011872610221096852, you)","(0.00027850702481264575, ribeiro)","(0.0030036594269634747, dr)","(0.0035284953741203727, deus)","(0.0013403844062696258, dra)"
3,"(0.001195028150291711, dr)","(0.0003750413322178258, 15)","(5.658907675476069e-05, sintomas)","(0.00010430143558720598, boa)","(0.00022635630701904277, virus)","(0.0026730016843572258, virus)","(0.0034996460408728476, olfato)","(0.0010663157404181378, tomar)"
4,"(0.0010097266636633771, comprimidos)","(0.00035284953741203726, comprimidos)","(5.436989727418184e-05, fiz)","(8.876717922315403e-05, noite)","(0.0001697672302642821, sobre)","(0.0026730016843572258, video)","(0.003411988451389983, paladar)","(0.0009287266126222491, deus)"
5,"(0.0008366306641782267, deus)","(0.0003428632297494324, tomar)","(5.436989727418184e-05, dor)","(8.765758948286461e-05, parabens)","(0.00014535625597791472, deus)","(0.002384508351881975, pessoas)","(0.0026796592227989625, dor)","(0.0009220690741805125, https)"
6,"(0.0007711648695011506, dia)","(0.00021747958909672737, cada)","(5.1041128053313566e-05, deu)","(7.545210233968093e-05, pinto)","(0.00014202748675704645, pra)","(0.002346782300712135, vc)","(0.0025487276334448103, pra)","(0.0008965485101538557, covid)"
7,"(0.0007556306131370987, pode)","(0.00021526040961614853, dose)","(4.4383589611577015e-05, deus)","(7.323292285910208e-05, gomes)","(0.00012094528169154737, tudo)","(0.0021526040961614853, vai)","(0.0022447000446055077, sinto)","(0.0008887813819718298, lucy)"
8,"(0.0006901648184600226, pra)","(0.00018530148662833403, kg)","(3.883564091012989e-05, positivo)","(6.54657946770761e-05, good)","(0.00011761651247067909, saude)","(0.0018707683021279712, agora)","(0.0022236178395400085, sentir)","(0.0008754663050883566, pra)"
9,"(0.0006846168697585755, cada)","(0.00018419189688804463, alvaro)","(3.883564091012989e-05, ainda)","(5.991784597562897e-05, the)","(0.00010430143558720598, video)","(0.0016699325591355851, casa)","(0.002183672608889589, gosto)","(0.0007467538952147833, comprimidos)"


In [6]:
#Selects only the N with higher std
word_std = X.std(axis=0)
higher_std_zipped = sorted(zip(word_std, labels, X.T), reverse=True)[:N]
word_std, labels, X_t = zip(*higher_std_zipped)

word_std_i = ['%.4f'%(x) for x in word_std]
print(f'{N} palavras com maior desvio padrão entre os canais:\n')
print(list(zip(labels, word_std_i)))

30 palavras com maior desvio padrão entre os canais:

[('dias', '0.0016'), ('pra', '0.0012'), ('cheiro', '0.0012'), ('olfato', '0.0011'), ('paladar', '0.0011'), ('drauzio', '0.0011'), ('deus', '0.0011'), ('dr', '0.0009'), ('vc', '0.0009'), ('virus', '0.0009'), ('dor', '0.0009'), ('video', '0.0008'), ('vai', '0.0008'), ('pessoas', '0.0007'), ('ivermectina', '0.0007'), ('sinto', '0.0007'), ('sentir', '0.0007'), ('nada', '0.0007'), ('gosto', '0.0007'), ('to', '0.0007'), ('dia', '0.0007'), ('assim', '0.0007'), ('sintomas', '0.0007'), ('agora', '0.0006'), ('covid', '0.0006'), ('bem', '0.0006'), ('ainda', '0.0006'), ('casa', '0.0005'), ('ta', '0.0005'), ('tudo', '0.0005')]


#### 2. Em relação ao canal
Foi obtida a distribuição de probabilidade das palavras em cada canal $C_k$, i.e., $p(a_i|C_k) = P(A=a_i|C_k)$. 
Com isso, podemos fazer corretamente comparações entre os canais. Foram, então, selecionadas as palavras cuja ocorrência mais se difere entre eles.

In [26]:
X, labels = word_occurence_matrix(comments_by_channel, stop_words=stop_words, binary=False)
#Makes frequencies relative to each channel so that we have the probability distribution of words for each channel
X = (X.T / X.sum(axis=1)).T

freq_lists = [list(zip(row,labels)) for row in X]
freq_lists = [sorted(channel, reverse=True) for channel in freq_lists]
df_freq = pd.DataFrame(freq_lists, index = channels).T

print('Palavras com maior frequência relativa em cada canal:')
df_freq.head(10)

Palavras com maior frequência relativa em cada canal:


,Dr. Alain Dutra,Dr. Alvaro Galvão,Dr. Felipe Ades MD PhD,Dr. Fernando Gomes,Dr. Lair Ribeiro Oficial,Drauzio Varella,Julio Pereira - Neurocirurgião,Lucy Kerr
0,"(0.016592081310679612, dias)","(0.017197143055266626, dias)","(0.02491961414790997, dias)","(0.04000613120784795, dr)","(0.023265001536368027, lair)","(0.009238836151382308, drauzio)","(0.017582739541819575, dias)","(0.013681406499733617, ivermectina)"
1,"(0.011974742111650486, tomar)","(0.014354066985645933, dr)","(0.01420150053590568, dia)","(0.03479460453709381, fernando)","(0.02247486940871779, dr)","(0.008942092076321352, pra)","(0.011952206199972204, cheiro)","(0.009511632036938377, dias)"
2,"(0.01149120145631068, ivermectina)","(0.013175230566534914, ivermectina)","(0.01420150053590568, covid)","(0.01640098099325567, you)","(0.011017953557789386, ribeiro)","(0.00828130114628871, dr)","(0.01194469381392571, deus)","(0.00858106908186823, dra)"
3,"(0.010211240898058253, dr)","(0.01171902087233895, 15)","(0.013665594855305467, sintomas)","(0.014408338442673207, boa)","(0.008954830780035995, virus)","(0.007369654400225158, virus)","(0.011847032795321287, olfato)","(0.006826496181850471, tomar)"
4,"(0.008627882281553399, comprimidos)","(0.011025587684626586, comprimidos)","(0.013129689174705252, fiz)","(0.012262415695892091, noite)","(0.006716123085026996, sobre)","(0.007369654400225158, video)","(0.011550293546484767, paladar)","(0.005945657964837507, deus)"
5,"(0.007148816747572816, deus)","(0.010713542750156022, tomar)","(0.013129689174705252, dor)","(0.01210913549969344, parabens)","(0.005750406040121153, deus)","(0.006574257910371053, pessoas)","(0.009071206151141695, dor)","(0.0059030367607884925, https)"
6,"(0.006589426577669903, dia)","(0.006795645239581166, cada)","(0.01232583065380493, deu)","(0.010423053341508276, pinto)","(0.0056187173521794476, pra)","(0.006470244523236285, vc)","(0.00862797537439854, pra)","(0.005739655478600604, covid)"
7,"(0.006456689927184466, pode)","(0.00672630192080993, dose)","(0.010718113612004287, deus)","(0.010116492949110975, gomes)","(0.004784688995215311, tudo)","(0.005934881501219098, vai)","(0.00759877848602884, sinto)","(0.00568993074054342, lucy)"
8,"(0.005897299757281553, pra)","(0.005790167117398238, kg)","(0.009378349410503752, positivo)","(0.009043531575720416, good)","(0.004653000307273605, saude)","(0.00515784031497701, agora)","(0.007527410818587146, sentir)","(0.0056046883324453916, pra)"
9,"(0.005849893810679611, cada)","(0.00575549545801262, alvaro)","(0.009378349410503752, ainda)","(0.00827713059472716, the)","(0.004126245555506782, video)","(0.004604121989347805, casa)","(0.007392187869750251, gosto)","(0.00478067838749778, comprimidos)"


In [8]:
#Selects only the N with higher std
word_std = X.std(axis=0)
higher_std_zipped = sorted(zip(word_std, labels, X.T), reverse=True)[:N]
word_std, labels, X_t = zip(*higher_std_zipped)

word_std_i = ['%.4f'%(x) for x in word_std]
print(f'{N} palavras com maior desvio padrão entre os canais:\n')
print(list(zip(labels, word_std_i)))

30 palavras com maior desvio padrão entre os canais:

[('dr', '0.0116'), ('fernando', '0.0115'), ('dias', '0.0083'), ('lair', '0.0077'), ('ivermectina', '0.0060'), ('you', '0.0054'), ('dor', '0.0046'), ('tomar', '0.0044'), ('fiz', '0.0042'), ('comprimidos', '0.0042'), ('sintomas', '0.0041'), ('olfato', '0.0041'), ('boa', '0.0040'), ('paladar', '0.0040'), ('cheiro', '0.0039'), ('deu', '0.0038'), ('covid', '0.0037'), ('noite', '0.0037'), ('15', '0.0036'), ('ribeiro', '0.0036'), ('parabens', '0.0035'), ('pinto', '0.0034'), ('dia', '0.0033'), ('gomes', '0.0033'), ('drauzio', '0.0030'), ('good', '0.0030'), ('positivo', '0.0029'), ('dra', '0.0029'), ('deus', '0.0028'), ('to', '0.0028')]
